<a href="https://colab.research.google.com/github/pratik-poudel/jane-street/blob/main/7%20lgbm%20regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets download ptrikp/js-parquet -f train.parquet
import zipfile
zip_ref = zipfile.ZipFile('/content/train.parquet.zip', 'r')
zip_ref.extractall('files')
zip_ref.close()

100% 1.07G/1.08G [00:19<00:00, 56.0MB/s]
100% 1.08G/1.08G [00:19<00:00, 57.9MB/s]


In [2]:
import numpy as np
import pandas as pd
from collections import defaultdict
from tqdm.notebook import tqdm
from matplotlib import pyplot as plt
import warnings
warnings.filterwarnings('ignore')
import seaborn as sns
import gc
# pd.options.display.float_format| = "{:.2f}".format
pd.options.display.max_columns = 500

In [3]:
train = pd.read_parquet('/content/files/train.parquet')
train = train.query('date > 85').reset_index(drop = True) 
train = train[train['weight'] != 0]
train.shape

(1571415, 138)

In [4]:
train.fillna(train.mean(), inplace=True)

In [8]:
features = [f for f in train.columns if 'feature' in f]
df_feature = []
df_corr = []
df_power = []
df_orig_corr=[]
for j in features:
    cor_list = []
    power = []
    orig_corr = train[j].corr(train['resp'])
    df_orig_corr.append(abs(orig_corr))
    for i in range(2, 7):
        cor = np.power(train[j], i).corr(train['resp'])
#         print(j)
        cor_list.append(abs(cor))
        power.append(i)
#         print(i, cor)
#     print(cor_list, power)
    index_cor = np.argmax(cor_list)
    df_feature.append(j)
    df_corr.append(cor_list[index_cor])
    df_power.append(power[index_cor])

In [9]:
power_df=pd.concat([pd.DataFrame(df_feature),pd.DataFrame(df_orig_corr), pd.DataFrame(df_corr), pd.DataFrame(df_power)], axis=1)
power_df.columns = ['feature', 'orig_corr', 'derived_corr', 'power']

In [118]:
power_df.to_csv('power_df.csv', index=False)

In [5]:
power_df = pd.read_csv('power_df.csv')

In [6]:
greater_005 = power_df[power_df['derived_corr'] > 0.005]

In [7]:
greater_than_orig = greater_005[(greater_005['derived_corr'] > greater_005['orig_corr'])]

In [8]:
df = pd.DataFrame()
for i, j in greater_than_orig[['feature', 'power']].values:
    df['power_'+str(i)+'_'+str(j)] = np.power(train[i], j)

In [24]:
df_cols = train[[f for f in train.columns if 'feature' in f]].corrwith(train['resp']).abs().sort_values(ascending=False)[:16].index.to_list()

In [25]:
for i in df_cols:
    df[i] = train[i]

In [27]:
df['resp'] = train['resp']
df['date'] = train['date']

In [28]:
val = df[df['date'] > 457]
tr = df[df['date'] <= 457]
val.shape, tr.shape

((197328, 34), (1374087, 34))

In [29]:
x_train = tr.drop(columns=['date', 'resp'])
y_train = tr['resp']
x_val = val.drop(columns=['date', 'resp'])
y_val= val['resp']
x_train.shape, y_train.shape, x_val.shape, y_val.shape

((1374087, 32), (1374087,), (197328, 32), (197328,))

In [40]:
import lightgbm as lgb
train_set = lgb.Dataset(x_train, y_train)
val_set = lgb.Dataset(x_val, y_val)
evals_result = {}  # to record eval results for plotting
params = {'n_estimators':5000,
                'boosting_type': 'gbdt',
                'objective': 'regression',
                'metric': 'rmse',
                'subsample': 0.75,
                'subsample_freq': 1,
                'learning_rate': 0.01,
                'feature_fraction': 0.9,
                'max_depth': 15,
                'lambda_l1': 1,  
                'lambda_l2': 1,
                'early_stopping_rounds': 100
                }
model = lgb.train(params, train_set, 
                    valid_sets=[train_set, val_set], verbose_eval = 100,  evals_result=evals_result)

Training until validation scores don't improve for 100 rounds.
[100]	training's rmse: 0.0268593	valid_1's rmse: 0.0269896
[200]	training's rmse: 0.0267631	valid_1's rmse: 0.0269648
[300]	training's rmse: 0.0266924	valid_1's rmse: 0.0269496
[400]	training's rmse: 0.0266336	valid_1's rmse: 0.026938
[500]	training's rmse: 0.0265821	valid_1's rmse: 0.0269319
[600]	training's rmse: 0.0265379	valid_1's rmse: 0.0269291
[700]	training's rmse: 0.0264963	valid_1's rmse: 0.0269262
[800]	training's rmse: 0.0264592	valid_1's rmse: 0.0269262
Early stopping, best iteration is:
[769]	training's rmse: 0.0264702	valid_1's rmse: 0.0269247


In [41]:
%%time
reg_preds = model.predict(df.drop(columns=['date', 'resp']))

CPU times: user 1min 25s, sys: 41.2 ms, total: 1min 25s
Wall time: 21.9 s


In [58]:
pd.Series(model.predict(x_val)).var()

6.656721907817594e-06

In [62]:
abs(np.corrcoef(y_val.values, model.predict(x_val))[0][1])

0.10657552929952924

In [51]:
pd.Series(model.predict(x_val)).corr(y_val)

nan

In [44]:
train['resp'].values[:10]

array([-0.02143543, -0.00474299,  0.0015269 , -0.00166304,  0.01712119,
       -0.00076449, -0.00138518, -0.00820158,  0.00392541,  0.00848051],
      dtype=float32)

In [65]:
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials, space_eval

In [99]:
def acc_model(params):                                                                                               
    params = {
        'objective': 'regression',
       'metric': 'rmse',
       'n_estimators':5000,
        'learning_rate': "{:.3f}".format(params['learning_rate']),
        'boosting_type': 'gbdt',
        'num_leaves': int(params['num_leaves']),
        'feature_fraction': '{:.3f}'.format(params['feature_fraction']),
        'bagging_fraction': '{:.3f}'.format(params['bagging_fraction']),
        'lambda_l1': "{:.3f}".format(params['lambda_l1']),
        'lambda_l2': "{:.3f}".format(params['lambda_l2']),
        'max_depth': int(params['max_depth']),
        'min_split_gain': "{:.3f}".format(params['min_split_gain']),
        'min_sum_hessian_in_leaf':"{:.3f}".format(params['min_sum_hessian_in_leaf']),
        'bagging_freq':int(params['bagging_freq']),
        # 'min_data_in_leaf': int(params['min_data_in_leaf']),
        # 'max_bin': int(params['max_bin'])
    }
   
    train_set = lgb.Dataset(x_train, y_train)
    val_set = lgb.Dataset(x_val, y_val)
    model = lgb.train(params, train_set, 
                    valid_sets=[train_set, val_set], verbose_eval = None, early_stopping_rounds=100)
    preds = model.predict(x_val)
    cor = np.corrcoef(y_val.values, preds)[0][1]
    cor = abs(cor)
    print("This is correlation between preds and actual ressp: ", cor)
    return -cor

In [103]:
# Defining Another space to check if optimization occurs
space={ 'objective': 'regression',
        'metric': 'rmse',
       'n_estimators': 5000,
        'learning_rate': hp.choice('learning_rate',[0.04, 0.05, 0.07, 0.1, 0.15, 0.2]),
        'boosting_type': 'gbdt',
        'num_leaves' : hp.choice('num_leaves',list(range(58, 125, 5))),
        'feature_fraction': hp.choice('feature_fraction',  [0.6, 0.7,0.8,0.9,1.0]),
        'bagging_fraction': hp.choice('bagging_fraction', [1.0]),
        'lambda_l1' : hp.choice('lambda_l1',[0]),
        'lambda_l2' : hp.choice('lambda_l2', [0, 0.001, 0.01, 0.1]),
        'max_depth' : hp.choice('max_depth',[5,10,15,20,25,30,35,40]),
        'min_split_gain': hp.choice('min_split_gain',[0.001, 0.01]),
        'min_sum_hessian_in_leaf' : hp.choice('min_sum_hessian_in_leaf', [0.001, 0.01]),
        'bagging_freq' : hp.choice('bagging_freq',[3, 5]),
        # 'min_data_in_leaf': hp.choice('min_data_in_leaf', [1, 10, 20, 30, 50, 70, 100, 120, 150]),
            # 'max_bin': hp.choice('max_bin', [63, 127, 255, 511] ),
    }

In [104]:
max_evals = 20
best = 0
def f(params):
    global best
    acc = acc_model(params)
    if acc > best:
        best = acc
    return {'loss': acc, 'status': STATUS_OK}

trials = Trials()
best = fmin(f, space, algo=tpe.suggest, max_evals=max_evals, trials=trials, rstate= np.random.RandomState(60)) 
best_results = space_eval(space, trials.argmin)
best_results['max_depth']=int(best['max_depth'])

This is correlation between preds and actual ressp: 
0.07724644955975096
This is correlation between preds and actual ressp: 
0.08978571753892149
This is correlation between preds and actual ressp: 
0.09444155165490739
This is correlation between preds and actual ressp: 
0.07875843756890634
This is correlation between preds and actual ressp: 
0.08376143846978575
This is correlation between preds and actual ressp: 
0.08001921120573016
This is correlation between preds and actual ressp: 
0.0880070261336754
This is correlation between preds and actual ressp: 
0.08706502492451641
This is correlation between preds and actual ressp: 
0.1029731945154208
This is correlation between preds and actual ressp: 
0.08025484533768486
This is correlation between preds and actual ressp: 
0.07657993114794527
This is correlation between preds and actual ressp: 
0.08737282871678727
This is correlation between preds and actual ressp: 
0.07294857323553218
This is correlation between preds and actual ressp: 


In [105]:
best_results

{'bagging_fraction': 1.0,
 'bagging_freq': 5,
 'boosting_type': 'gbdt',
 'feature_fraction': 0.8,
 'lambda_l1': 0,
 'lambda_l2': 0.01,
 'learning_rate': 0.07,
 'max_depth': 0,
 'metric': 'rmse',
 'min_split_gain': 0.001,
 'min_sum_hessian_in_leaf': 0.01,
 'n_estimators': 5000,
 'num_leaves': 123,
 'objective': 'regression'}

In [31]:
import lightgbm as lgb
train_set = lgb.Dataset(x_train, y_train)
val_set = lgb.Dataset(x_val, y_val)

def objective(trial):    
    params = {
            'num_leaves': trial.suggest_int('num_leaves', 4, 512),
            'boosting_type': 'gbdt',
            'max_bin': trial.suggest_int('max_bin', 50, 900),
            'objective': 'regression',
            'metric': 'rmse',
            'max_depth': trial.suggest_int('max_depth', 4, 512),
            'learning_rate':trial.suggest_uniform('learning_rate', 0.01, 0.9),
            'min_child_weight': trial.suggest_int('min_child_weight', 1, 16),
            'feature_fraction': trial.suggest_uniform('feature_fraction', 0.4, 1.0),
            'bagging_fraction': trial.suggest_uniform('bagging_fraction', 0.4, 1.0),
            'bagging_freq': trial.suggest_int('bagging_freq', 1, 8),
            'min_child_samples': trial.suggest_int('min_child_samples', 4, 100),
            'lambda_l1': trial.suggest_loguniform('lambda_l1', 1e-8, 1.0),
            'lambda_l2': trial.suggest_loguniform('lambda_l2', 1e-8, 1.0),
            'early_stopping_rounds': 100
            }

    
    model = lgb.train(params, train_set, 
                    valid_sets=[train_set, val_set], verbose_eval = None, early_stopping_rounds=100)
    preds = model.predict(x_val)
    cor = np.corrcoef(y_val.values, preds)[0][1]
    cor = abs(cor)
    # print("This is correlation between preds and actual ressp: ", cor)
    return cor

In [12]:
!pip install optuna

In [ ]:
import optuna
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=150)
print('Number of finished trials:', len(study.trials))
print('Best trial:', study.best_trial.params)

[I 2021-02-22 10:33:34,246] A new study created in memory with name: no-name-41ed3a7b-e4e3-425e-9a6b-5c74a020bf53
[I 2021-02-22 10:34:06,195] Trial 0 finished with value: 0.08325627328141368 and parameters: {'num_leaves': 375, 'max_bin': 385, 'max_depth': 420, 'learning_rate': 0.08443757451771348, 'min_child_weight': 3, 'feature_fraction': 0.5999540770296483, 'bagging_fraction': 0.9059863523170117, 'bagging_freq': 8, 'min_child_samples': 19, 'lambda_l1': 0.0004992434130278159, 'lambda_l2': 1.2449738085456017e-05}. Best is trial 0 with value: 0.08325627328141368.
[I 2021-02-22 10:34:54,507] Trial 1 finished with value: 0.0470532340689203 and parameters: {'num_leaves': 489, 'max_bin': 426, 'max_depth': 215, 'learning_rate': 0.43973257399645715, 'min_child_weight': 8, 'feature_fraction': 0.8571697237689362, 'bagging_fraction': 0.7291950090289263, 'bagging_freq': 5, 'min_child_samples': 69, 'lambda_l1': 0.2709847036085578, 'lambda_l2': 8.384609833580355e-08}. Best is trial 0 with value: 0.

In [13]:
def objective(trial):
    param = {
        'tree_method':'gpu_hist',  # this parameter means using the GPU when training our model to speedup the training process
        'lambda': trial.suggest_loguniform('lambda', 1e-3, 10.0),
        'alpha': trial.suggest_loguniform('alpha', 1e-3, 10.0),
        'colsample_bytree': trial.suggest_categorical('colsample_bytree', [0.3,0.4,0.5,0.6,0.7,0.8,0.9, 1.0]),
        'subsample': trial.suggest_categorical('subsample', [0.4,0.5,0.6,0.7,0.8,1.0]),
        'learning_rate': trial.suggest_categorical('learning_rate', [0.008,0.009,0.01,0.012,0.014,0.016,0.018, 0.02]),
        'n_estimators': 5000,
        'max_depth': trial.suggest_int('max_depth', 4, 32),
        'random_state': trial.suggest_int('random_state', 1, 2222),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 300),
    }
    model = xgb.XGBRegressor(**param)  
    
    model.fit(x_train,y_train,eval_set=[(x_val,y_val)],early_stopping_rounds=100,verbose=False)
    
    preds = model.predict(x_val)
    cor = np.corrcoef(y_val.values, preds)[0][1]
    cor = abs(cor)
        
    return cor

In [15]:
import xgboost as xgb
import optuna
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=250)
print('Number of finished trials:', len(study.trials))
print('Best trial:', study.best_trial.params)

[I 2021-02-22 10:25:00,844] A new study created in memory with name: no-name-d8bed95b-73f6-4eff-b0a8-ec824342e0a0


[10:25:01] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


[I 2021-02-22 10:25:08,263] Trial 0 finished with value: 0.09705275537684314 and parameters: {'lambda': 0.12323300596394882, 'alpha': 0.5223920114890946, 'colsample_bytree': 0.9, 'subsample': 0.4, 'learning_rate': 0.02, 'max_depth': 13, 'random_state': 1098, 'min_child_weight': 239}. Best is trial 0 with value: 0.09705275537684314.


[10:25:08] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


[I 2021-02-22 10:28:14,837] Trial 1 finished with value: 0.09045275392700719 and parameters: {'lambda': 2.8358524580720195, 'alpha': 9.35346230644765, 'colsample_bytree': 0.4, 'subsample': 0.7, 'learning_rate': 0.008, 'max_depth': 24, 'random_state': 227, 'min_child_weight': 162}. Best is trial 0 with value: 0.09705275537684314.


[10:28:15] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


KeyboardInterrupt: ignored